In [741]:
%%capture
%run 01_transaction.ipynb
from IPython.display import Image

In [742]:
from random import randint as ri

Create a random tx.

In [743]:
def rt(): return TX(rh(),rh(),ri(1,99),ri(1,99)/100,0); 
print(rt())

time:	Mon Mar 29 00:58:55 2021
from:	👿 1927cbbc4013b9c4...a76
to:	📔 6e1e942171cc24fa...4c7
value:	90 ether
fee:	0.05 ether
nonce:	0
hash:	👱 0be7b9d9a6fe922b...f9e
signed:	false


## Merkle Tree

A binary tree in which every leaf node is a hash of a specific piece of data, in our case a tx. 

In a system like Bitcoin or Ethereum the merkle root is included in the block header, not the list of txs itself. This makes it very efficient to test if a specific tx is inclued in the tree.

For more in depth [here](https://www.youtube.com/watch?v=3giNelTfeAk).

<div>
<img src="https://changelly.com/blog/wp-content/uploads/2020/01/Merkle-Tree-1.png" width="500"/>
</div>


`MerkleTree` is a wrapper around the `mt` dictionary which stores the hashes for each height in the tree.

In [744]:
class MerkleTree:
    def __init__(self, txs):
        self.mt = {}
        leafs = [tx.hash for tx in txs]
        if len(leafs)%2!=0: leafs.append(sha('0x0'))
        self.mt['1'] = leafs
        self.merkle(leafs)
        
    def merkle(self, leafs):
        parents  = []
        while len(leafs) > 1:
            for i in range(0, len(leafs), 2):
                l = leafs[i]
                if i+1>=len(leafs): r = '0x0'
                else              : r = leafs[i+1]
                parents.append(sha(l+r))
            leafs = parents
            self.mt[f'{len(self)+1}'] = parents
            parents = []
    
    @property
    def root(self)      : return self.mt[str(len(self))][0]
    def __eq__ (self, o): return self.root == o.root
    def __len__(self)   : return len(self.mt)
    def __getitem__(self, k): return self.mt[str(k)]
    def __str__(self):
        s = ''
        for k,v in self.mt.items(): 
            s += f'height {k}'
            for h in v: s += f'\n {ph(h)}'
            s += '\n'
        return s

Create Merkle Tree from a list of `N` random txs.

In [745]:
N = 8
r_txs = [rt() for _ in range(N)]

mt = MerkleTree(r_txs); print(mt)

height 1
 💁 1bbb73466713e0ef...c10
 💁 1b8e3221549c36d1...5fc
 🕝 f7bf4a3df716acc5...556
 💧 412e15d846edbe84...82b
 🔔 ae2d7b9d75f7226f...4e2
 💋 25d5e7f83788bff2...2d7
 🕘 f2fc28ff8ff8fd08...039
 📾 980f963d5d9b0574...b48
height 2
 🔉 a3a7aa5b095d17ed...fe1
 🔒 ac82f385dd914a4e...afd
 📒 6ca688c1018542e0...2d3
 👴 0e562829ddfeb296...548
height 3
 📁 5b366155b12c41d0...23a
 💃 1df40ece66f30da8...fa4
height 4
 🔜 b65f468e79a85c79...311



In [746]:
assert len(mt) == 4

The merkle tree root hash is the top most node. If anything in the tree changes the root changes as well.

In [747]:
ph(mt.root)

'🔜 b65f468e79a85c79...311'

If we change something in any tx the whole Merkle Tree changes. Here we change the `value` of the third tx. Compare the hashes below with the ones above.

In [748]:
r_txs[2].value = 500

changed_mt = MerkleTree(r_txs); print(changed_mt)

height 1
 💁 1bbb73466713e0ef...c10
 💁 1b8e3221549c36d1...5fc
 🔇 a1993d78b74e64e4...222
 💧 412e15d846edbe84...82b
 🔔 ae2d7b9d75f7226f...4e2
 💋 25d5e7f83788bff2...2d7
 🕘 f2fc28ff8ff8fd08...039
 📾 980f963d5d9b0574...b48
height 2
 🔉 a3a7aa5b095d17ed...fe1
 💺 54e09ae080e74319...cff
 📒 6ca688c1018542e0...2d3
 👴 0e562829ddfeb296...548
height 3
 📜 764b6d879560e58a...afc
 💃 1df40ece66f30da8...fa4
height 4
 💅 1f82f626432c33ac...06c



In [749]:
assert mt != changed_mt

## Merkle Proof

Proof that `tx` at position `pos` in the merkle tree `mt` is part of it by using a corresponding merkle branch `mb`.

A merkle branch consists of those nodes that are needed to build up the root hash of the merkle tree. For example:

Merkle branch `[5,3,0, ..., x]` stands for the following: 
- node *5* from height *0*
- node *3* from height *1*
- node *0* from height *2*
- ...
- node *x* from height `len(mt)-1`

In [750]:
def proof(mt, mb, tx):
    if mb[0]%2 != 0: cn = sha(tx.hash     +mt[1][mb[0]])
    else           : cn = sha(mt[1][mb[0]]+tx.hash)
    for i in range(2, len(mt)):
        if mb[i-1]%2 != 0: cn = sha(cn            +mt[i][mb[i-1]])
        else             : cn = sha(mt[i][mb[i-1]]+cn)
    return cn == mt.root

Examples of some txs with their corresponding merkle branch.

In [751]:
r_tx = r_txs[0]
mb = [1,1,1]

assert proof(mt, mb, r_tx)

In [752]:
r_tx = r_txs[1]
mb = [0,1,1]

assert proof(mt, mb, r_tx)

In [753]:
r_tx = r_txs[4]
mb = [5,3,0]

assert proof(mt, mb, r_tx)